- References
    - ["19-04 BERT를 이용한 키워드 추출 : 키버트(KeyBERT)", 딥 러닝을 이용한 자연어 처리 입문 (2022)](https://wikidocs.net/159467)
    - [ukairia777 GitHub - tensorflow-nlp-tutorial/19. Topic Modeling (LDA, BERT-Based)
/19-5. keybert_kor.ipynb](https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/df3f84702cd4e00ec2319549a2f029c3b2d666f6/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/19-5.%20keybert_kor.ipynb)

In [1]:
# !pip install sentence_transformers

In [2]:
HUGGINGFACE_MODEL_PATH = 'jhgan/ko-sroberta-multitask'

특정 문서의 주요 정보를 이해하고자 할 때 키워드 추출을 통해서 입력 텍스트와 가장 관련이 있는 단어와 구문을 추출할 수 있습니다. 'Rake'나 'YAKE!'와 같은 키워드를 추출하는 데 사용할 수 있는 패키지가 이미 존재합니다. 그러나 이러한 모델은 일반적으로 텍스트의 통계적 특성에 기반하여 작동하며 의미적인 유사성에 대해서는 고려하지 않습니다. 의미적 유사성을 고려하기 위해서 여기서는 SBERT 임베딩을 활용하여 사용하기 쉬운 키워드 추출 알고리즘인 KeyBERT를 사용합니다.

# 1. 기본 KeyBERT

In [3]:
import numpy as np
import itertools
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [4]:
def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

In [5]:
def tokenizer(string):
    return string.split(" ")

In [6]:
doc = """
B737-700 항공기 항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게 설정하도록 지시를 받았다. 그러한 복행 기동은 복잡한 사안, 통찰력, 항공 우수성에도 전하여 쟁취하려는 항공 관리 철학 등이 뒤엉킨 총체적 난국과도 같은 상황을 야기 했다. 기장이 작성한 보고서에는 다음과 같이 기술되어 있다. ■접근관제소가 우리를 너무 가깝고 높게 유도하여 ILS 상에서 적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며, 저는 부조종사에게 '1,000피트 정도 차이로 실패하게 될 것 입니다. '라고 말했습니다. 우리는 3,000피트 MSL에 위치해 있었고 조종 승무원은 다급히 플랩을 30도로 설정하고 있었습니다. 저는 접근관제소에게 '우리는 복행 기동을 실시하겠다'라고 말했습니다. 조종 승무원이 출력을 증가시켰지만, 항공기 노즈를 너무 높게 피치 업 시켜서 결국 'AIRSPEED' 경고가 발생하게 되었습니다. PF가 항공기 노즈를 낮춰서 피치를 어느 정도 바로잡기는 했지만 노즈가 다시 피치 업 되면서 두 번째 'AIRSPEED' 경고가 발생 하였습니다. 첫 번째 'AIRSPEED' 경고가 발생하자 저는 플랩 레버를 15도(제 생각에는)로 맞추고 있었습니다. 두 번째 경고가 발생했을 때 PF는 조종간을 힘껏 아래로 내려 비행 속도를 다시 증가시키려 했습니다. PF가 이러한 보다 과격한 기동을 사용하자, 저는 '수평을 유지 하세요'라고 말했습니다. 그는 수평을 유지했고 비행 속도도 증가하기 시작했습니다. 우리는 착륙 진입을 다시 시도하기 위해 진로를 다시 잡았고, 이번에는 아무런 사고도 없이 무사히 착륙할 수 있었습니다.

[주요 원인]
PF 및 내 자신의 조종 실수, 기장으로서 조종 제어를 맡지 않으려 한 점과 지극히 이례적이고 예측할 수 없었던 극적인 복행 기동 상황에서 적절할 조언을 제공할 준비가 되어 있지 않았던 점 등 부기장이 작성한 보고서에는 다음과 같이 기술되어 있다.
■착륙 진입을 계속 시도하지 않았어야 했었습니다. Glideslope가 너무 낮다는 점이 명확하게 드러났을 때 제가 즉시 복행 기동을 실시했었어야 했습니다. 두 승무원들 사이의 교류가 단절되었던 점은 분명합니다. 한 명은 항공기를 조종하고 있었고 다른 한 명은 ATC와 교신하고 있었습니다. 복행 기동은 적시에 승무원 모두의 협력을 통해 이루어진 것이 아니었고, 그로 인해 오류가 발생하게 되었습니다. PM과 PF 사이의 조화도 이루어지지 않았습니다. 우리는 복행 기동을 자주 시도하는 편이 아니기 때문에 익숙하지 않을 수 있습니다. 우리는 차트에 복행 절차에 대한 과정을 작성해 놓긴 하지만, 머릿속으로 또는 구두로 해당 기동에 대해 점검해 보아야 할 것입니다.
"""

In [7]:
processed_doc = process_doc(doc)

In [8]:
tokenized_doc = " ".join(processed_doc.split(' '))
tokenized_doc

"B737-700 항공기 항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게 설정하도록 지시를 받았다. 그러한 복행 기동은 복잡한 사안, 통찰력, 항공 우수성에도 전하여 쟁취하려는 항공 관리 철학 등이 뒤엉킨 총체적 난국과도 같은 상황을 야기 했다. 기장이 작성한 보고서에는 다음과 같이 기술되어 있다. ■접근관제소가 우리를 너무 가깝고 높게 유도하여 ILS 상에서 적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며, 저는 부조종사에게 '1,000피트 정도 차이로 실패하게 될 것 입니다. '라고 말했습니다. 우리는 3,000피트 MSL에 위치해 있었고 조종 승무원은 다급히 플랩을 30도로 설정하고 있었습니다. 저는 접근관제소에게 '우리는 복행 기동을 실시하겠다'라고 말했습니다. 조종 승무원이 출력을 증가시켰지만, 항공기 노즈를 너무 높게 피치 업 시켜서 결국 'AIRSPEED' 경고가 발생하게 되었습니다. PF가 항공기 노즈를 낮춰서 피치를 어느 정도 바로잡기는 했지만 노즈가 다시 피치 업 되면서 두 번째 'AIRSPEED' 경고가 발생 하였습니다. 첫 번째 'AIRSPEED' 경고가 발생하자 저는 플랩 레버를 15도(제 생각에는)로 맞추고 있었습니다. 두 번째 경고가 발생했을 때 PF는 조종간을 힘껏 아래로 내려 비행 속도를 다시 증가시키려 했습니다. PF가 이러한 보다 과격한 기동을 사용하자, 저는 '수평을 유지 하세요'라고 말했습니다. 그는 수평을 유지했고 비행 속도도 증가하기 시작했습니다. 우리는 착륙 진입을 다시 시도하기 위해 진로를 다시 잡았고, 이번에는 아무런 사고도 없이 무사히 착륙할 수 있었습니다. [주요 원인] PF 및 내 자신의 조종 실수, 기장으로서 조종 제어를 맡지 않으려 한 점과 지극히 이례적이고 예측할 수 없었던 극적인 복행 기동 상황에서 적절할 조언을 제공할 준비가 되어 있지 않았던 점 등 부기장이 작성한 보고서에는 다음과 같이 기술되어 있다. ■착륙 진입을 계속 시도하지 않았어야 했었습니다. Glideslope

여기서는 사이킷런의 CountVectorizer를 사용하여 단어를 추출합니다.  CountVectorizer를 사용하는 이유는 n_gram_range의 인자를 사용하면 단쉽게 n-gram을 추출할 수 있기 때문입니다. 예를 들어, (2, 3)으로 설정하면 결과 후보는 2개의 단어를 한 묶음으로 간주하는 bigram과 3개의 단어를 한 묶음으로 간주하는 trigram을 추출합니다.

In [9]:
n_gram_range = (3, 8)

count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=tokenizer, token_pattern=None).fit([tokenized_doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 1846
trigram 다섯개만 출력 : ["'1,000피트 정도 차이로" "'1,000피트 정도 차이로 실패하게" "'1,000피트 정도 차이로 실패하게 될"
 "'1,000피트 정도 차이로 실패하게 될 것" "'1,000피트 정도 차이로 실패하게 될 것 입니다."]


다음으로 이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화 할 차례입니다. 한국어를 포함하고 있는 다국어 SBERT를 로드합니다.

In [10]:
HUGGINGFACE_MODEL_PATH

'jhgan/ko-sroberta-multitask'

In [11]:
model = SentenceTransformer(HUGGINGFACE_MODEL_PATH)

In [12]:
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

이제 문서와 가장 유사한 키워드들을 추출합니다. 여기서는 문서와 가장 유사한 키워드들은 문서를 대표하기 위한 좋은 키워드라고 가정합니다. 상위 5개의 키워드를 출력합니다.

In [13]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

In [14]:
keywords

['기동 중 ATC로부터 고도를 높게 설정하도록 지시를 받았다.',
 '복행 기동 중 ATC로부터 고도를 높게 설정하도록 지시를',
 '승무원들은 복행 기동 중 ATC로부터 고도를 높게 설정하도록',
 '항공기 항공 승무원들은 복행 기동 중 ATC로부터 고도를',
 '항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게']

5개의 키워드가 출력되는데, 이들의 의미가 좀 비슷해보입니다. 비슷한 의미의 키워드들이 리턴되는 데는 이유가 있습니다. 당연히 이 키워드들이 문서를 가장 잘 나타내고 있기 때문입니다. 만약, 지금 출력한 것보다는 좀 더 다양한 의미의 키워드들이 출력된다면 이들을 그룹으로 본다는 관점에서는 어쩌면 해당 키워드들이 문서를 잘 나타낼 가능성이 적을 수도 있습니다. 따라서 키워드들을 다양하게 출력하고 싶다면 키워드 선정의 정확성과 키워드들의 다양성 사이의 미묘한 균형이 필요합니다.

여기서는 다양한 키워드들을 얻기 위해서 두 가지 알고리즘을 사용합니다.

* Max Sum Similarity  
* Maximal Marginal Relevance

# 2. Max Sum Similarity

데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [15]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

이를 위해 상위 10개의 키워드를 선택하고 이 10개 중에서 서로 가장 유사성이 낮은 5개를 선택합니다.

낮은 nr_candidates를 설정하면 결과는 출력된 키워드 5개는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [16]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['ATC로부터 고도를 높게 설정하도록 지시를 받았다. 그러한 복행',
 '항공기 항공 승무원들은 복행 기동 중 ATC로부터',
 'B737-700 항공기 항공 승무원들은 복행 기동 중 ATC로부터',
 '기동 중 ATC로부터 고도를 높게 설정하도록 지시를 받았다.',
 '항공기 항공 승무원들은 복행 기동 중 ATC로부터 고도를']

그러나 상대적으로 높은 nr_candidates는 더 다양한 키워드 5개를 만듭니다.

In [17]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['B737-700 항공기 항공 승무원들은 복행 기동',
 '그러한 복행 기동은 복잡한 사안, 통찰력, 항공',
 '중 ATC로부터 고도를 높게 설정하도록 지시를 받았다.',
 '위치해 있었고 조종 승무원은 다급히',
 '중 ATC로부터 고도를 높게 설정하도록 지시를 받았다. 그러한']

# 3. Maximal Marginal Relevance

결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [18]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

만약 우리가 상대적으로 낮은 diversity 값을 설정한다면, 결과는 기존의 코사인 유사도만 사용한 것과 매우 유사한 것으로 보입니다.

In [19]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며,',
 '기동 중 ATC로부터 고도를 높게 설정하도록 지시를 받았다.',
 '위치해 있었고 조종 승무원은 다급히',
 '받았다. 그러한 복행 기동은 복잡한 사안, 통찰력, 항공']

In [20]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.3)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며,',
 '높게 설정하도록 지시를 받았다. 그러한 복행 기동은 복잡한',
 '위치해 있었고 조종 승무원은 다급히',
 '쟁취하려는 항공 관리 철학 등이 뒤엉킨 총체적 난국과도']

In [21]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.4)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며,',
 '상황을 야기 했다. 기장이 작성한 보고서에는 다음과',
 '쟁취하려는 항공 관리 철학 등이 뒤엉킨 총체적 난국과도',
 '있었고 조종 승무원은 다급히 플랩을 30도로 설정하고 있었습니다.']

In [22]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.5)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '난국과도 같은 상황을 야기 했다. 기장이 작성한 보고서에는',
 '적절한 항로를 수립할 수 없었습니다. 저는 관제 승무원(PM)이었으며,',
 '높게 설정하도록 지시를 받았다.',
 '전하여 쟁취하려는 항공 관리 철학 등이 뒤엉킨 총체적']

In [23]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.6)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '수립할 수 없었습니다. 저는',
 '난국과도 같은 상황을 야기 했다. 기장이 작성한 보고서에는',
 "다급히 플랩을 30도로 설정하고 있었습니다. 저는 접근관제소에게 '우리는",
 '승무원들 사이의 교류가 단절되었던 점은']

그러나 상대적으로 높은 diversity값은 다양한 키워드 5개를 만들어냅니다.

In [24]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '수립할 수 없었습니다. 저는',
 '보고서에는 다음과 같이 기술되어 있다.',
 '15도(제 생각에는)로 맞추고',
 '승무원들 사이의 교류가 단절되었던 점은 분명합니다.']

In [25]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.8)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '수립할 수 없었습니다. 저는',
 'Glideslope가 너무 낮다는',
 '보고서에는 다음과 같이 기술되어 있다.',
 '그는 수평을 유지했고']

In [26]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.9)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '사이의 교류가 단절되었던 점은 분명합니다.',
 '및 내 자신의',
 'Glideslope가 너무 낮다는',
 '두 번째 경고가 발생했을 때 PF는']

In [27]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=1.0)

['항공 승무원들은 복행 기동 중 ATC로부터 고도를 높게',
 '사이의 교류가 단절되었던 점은 분명합니다.',
 '및 내 자신의',
 'Glideslope가 너무 낮다는',
 '두 번째 경고가 발생했을 때 PF는']

# 4. 중간 정리
결론적으로, 테스트 케이스에 대해서는 **Max Sum Similarity** (`nr_candidates=30`)의 성능이 가장 우수한 것을 확인하였습니다.

위 과정을 정리하여 **document를 입력으로 받고, keyphrase를 반환하는 함수**를 만들어 봅시다.

In [28]:
import numpy as np
import itertools
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

HUGGINGFACE_MODEL_PATH = 'jhgan/ko-sroberta-multitask'
model = SentenceTransformer(HUGGINGFACE_MODEL_PATH)

def process_doc(doc):
    processed_doc = doc.strip('"').replace("\n", " ").strip()
    processed_doc = re.sub('\s+', ' ', processed_doc)
    
    return processed_doc

def tokenizer(string):
    return string.split(" ")

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [29]:
def doc_to_keyphrases(doc, model=SentenceTransformer(HUGGINGFACE_MODEL_PATH), n_gram_range=(1,10), keyword_top_n=5, mss_nr_candidates=30):
    processed_doc = process_doc(doc)
    tokenized_doc = " ".join(processed_doc.split(' '))
    
    count = CountVectorizer(ngram_range=n_gram_range, lowercase=False, tokenizer=tokenizer, token_pattern=None).fit([tokenized_doc])

    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)

    # keyphrases_list = mmr(doc_embedding, candidate_embeddings, candidates, top_n=keyword_top_n, diversity=mmr_diversity)    
    keyphrases_list = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=keyword_top_n, nr_candidates=mss_nr_candidates)    
    
    return keyphrases_list

In [30]:
doc = """
아내에게 집에서 저녁식사를 하겠다고 하고, 집을 나선 헬리콥터 조종사는 '모두를 행복하게' 해줄 생각으로 집으로 돌아가기로 마음먹었지만 그 결정은 그다지 현명하지 못한 결정이 되어버렸다. 나는 헬리콥터를 집 뒤에 있는 마을 공터에 착륙시켰다. 공터 면적은 충분히 넓었고, 착륙은 교과서대로 진행하였으며, 착륙장소 주변엔 아무도 없었다. 같은 장소는 아니지만 전에도 아무 문제없이 착륙해 본 경험이 있어서 걱정 없이 착륙했는데. 착륙하고 보니 큰 일이 벌어져 있었다. 
지나가던 구급차가 내 헬리콥터가 착륙하는 장면을 보고는 경찰서에 추락신고를 한 것이다. 수 분만에 경찰차, 소방차, 기자들이 모여들었다. 
단지 가족과 저녁식사를 하려했다는 이유는 관계당국에 설득력 있는 이유가 되지 못했고 여러 가지 주정부 법에 따라 벌금형이 내려졌다. 
착륙 전 충분한 '주의'를 기울였음에도 불구하고 FAR91.13 조차도 위반했다고 한다.
"""

In [31]:
doc_to_keyphrases(doc, model=model)

["헬리콥터 조종사는 '모두를 행복하게' 해줄 생각으로 집으로 돌아가기로 마음먹었지만 그",
 "하고, 집을 나선 헬리콥터 조종사는 '모두를 행복하게' 해줄 생각으로 집으로",
 '그다지 현명하지 못한 결정이 되어버렸다. 나는 헬리콥터를 집 뒤에',
 '헬리콥터를 집 뒤에 있는 마을 공터에 착륙시켰다. 공터 면적은',
 '헬리콥터를 집 뒤에 있는 마을 공터에 착륙시켰다. 공터 면적은 충분히']

# 5. 테스트셋 추론

In [32]:
import pandas as pd
from tqdm import tqdm

In [33]:
df = pd.read_csv("../data/rawdata/GYRO_testset.csv")

In [34]:
predicted_keyphrases_list = []
for i in tqdm(range(len(df))):
    doc = df["본문"][i]
    predicted_keyphrase = doc_to_keyphrases(doc, model=model)
    predicted_keyphrases_list.append('"' + '", "'.join(predicted_keyphrase) + '"')

100% 219/219 [15:45<00:00,  4.32s/it]


In [35]:
df["KeyBERT 예측 키워드"] = predicted_keyphrases_list

In [37]:
df.to_csv("../data/prediction/KeySRoBERTa.csv", index=False)

In [38]:
exit()